# The Guardian API

In the `05_web_scraping_beautiful_soup.ipynb` notebook, we saw examples on how BeautifulSoup can be used 
to parse messy HTML, to extract information, and to act as a rudimentary web crawler. 
We used The Guardian as an illustrative example about how this can be achieved. 
The reason for choosing The Guardian was because they provide a REST API to their servers. 
With the REST API it is possible to perform specific queries on their servers, and to receive 
current information from their servers according to their API guide (ie in JSON)

http://open-platform.theguardian.com/

In order to use their API, you will need to register for an API key. 
At the time of writing (Jan 28, 2020) this was an automated process that can be completed at 

https://bonobo.capi.gutools.co.uk/register/developer

On registration you will receive an API key which will look like: 303qwe2k-xxxx-xxxx-xxxx-eff86a248059

The API is documented here: 

http://open-platform.theguardian.com/documentation/

and Python bindings to their API are provided by The Guardian here

https://github.com/prabhath6/theguardian-api-python

and these can easily be integrated into a web-crawler based on API calls, rather than being based 
on HTML parsing, etc. 

We use four parameters in our queries here: 

1. `section`: the section of the newspaper that we are interested in querying. In this case we will look at 
the technology section 

2. `order-by`: We have specified that the newest items should be closer to the front of the query list 

3. `api-key`: In this notebook, the api-key is left as `test` (works here), but for *real* deployment of such a spider an API key obtained from Guardian should be specified. For the lab tasks, you should replace `test` API key with your personal API key. 

4. `page-size`: The number of results to return. 

In [40]:
from __future__ import print_function

import requests 
import json 

from bs4 import BeautifulSoup

# Inspect all sections and search for technology-based sections

In [4]:
# base url, setting section to technology
url = 'https://content.guardianapis.com/search?api-key=cb2b9a64-0ab6-43a0-911f-3ee88d36c18e&section=technology'
req = requests.get(url)
src = req.text 

In [6]:
json.loads(src)['response']['status']

'ok'

In [7]:
sections = json.loads(src)['response']
print(sections.keys())

dict_keys(['status', 'userTier', 'total', 'startIndex', 'pageSize', 'currentPage', 'pages', 'orderBy', 'results'])


In [11]:
# Task 6

print(sections['total'],sections['pageSize'],sections['pages'])

53931 10 5394


In [8]:
# EXTRA USEFUL CODE, ALREADY HERE

print(json.dumps(sections['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/technology/2021/feb/04/facebook-groups-misinformation",
  "id": "technology/2021/feb/04/facebook-groups-misinformation",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2021-02-04T11:00:33Z",
  "webTitle": "'It let white supremacists organize': the toxic legacy of Facebook's Groups",
  "webUrl": "https://www.theguardian.com/technology/2021/feb/04/facebook-groups-misinformation"
}


In [6]:
# EXTRA USEFUL CODE, ALREADY HERE

for result in sections['results']: 
    if 'tech' in result['id'].lower(): 
        print(result['webTitle'], result['apiUrl'])

Technology https://content.guardianapis.com/technology


# Manual query on whole API

In [17]:
# Specify the arguments

# Task 7: News stories about a specific topic

args = {
    'section': 'technology', 
    'order-by': 'newest', 
    'api-key': 'cb2b9a64-0ab6-43a0-911f-3ee88d36c18e', 
    'page-size': '100',
    'q' : 'privacy%20AND%20signal%20AND%20whatsapp%20AND%20NOT%20ai%20AND%20NOT%20artificial%20intelligence'
}

# Construct the URL
base_url = 'http://content.guardianapis.com/search'
url = '{}?{}'.format(
    base_url, 
    '&'.join(["{}={}".format(kk, vv) for kk, vv in args.items()])
)

# Make the request and extract the source
req = requests.get(url) 
src = req.text

sections = json.loads(src)['response']

print(sections['total'])

34


In [27]:
# Task 8

i = 0

article = sections['results'][i]

api_key = 'cb2b9a64-0ab6-43a0-911f-3ee88d36c18e'

api_url = article['apiUrl']
api_id = article['id']

search_string = "ids=%s&api-key=%s&show-fields=headline,body" %(api_id, api_key)

url = base_url + '?' + search_string

print(url)

http://content.guardianapis.com/search?ids=technology/2021/jan/26/uk-regulator-to-write-to-whatsapp-over-facebook-data-sharing&api-key=cb2b9a64-0ab6-43a0-911f-3ee88d36c18e&show-fields=headline,body


In [41]:
req = requests.get(url) 
src = req.text

article = json.loads(src)['response']['results'][0]

body = article['fields']['body']

cleantext = BeautifulSoup(body, "lxml").text

print(cleantext)

The UK’s data regulator is writing to WhatsApp to demand that the chat app does not hand user data to Facebook, as millions worldwide continue to sign up for alternatives such as Signal and Telegram to avoid forthcoming changes to its terms of service. Elizabeth Denham, the information commissioner, told a parliamentary committee that in 2017, WhatsApp had committed not to hand any user information over to Facebook until it could prove that doing so respected GDPR. But, she said, that agreement was enforced by the Irish data protection authority until the Brexit transition period ended on 1 January. Now that Britain is fully outside the EU, ensuring that those promises are being kept falls to the Information Commissioner’s Office. “The change in the terms of service, and the requirement of users to share information with Facebook, does not apply to UK users or to users in the EU,” Denham told the digital, culture, media and sport sub-committee on online harms and disinformation, “and t

In [91]:
import re
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
from collections import Counter
d = dict(Counter(words))
df = pd.DataFrame(d.items())
import string

print(df)

words = re.findall("[a-zA-Z_]+", cleantext)

words2 = re.sub('['+string.punctuation+']', '', cleantext).split()

words3 = [i.strip(string.punctuation) for i in cleantext.split()]


print(len(words),len(set(words)))
print(len(words2),len(set(words2)))
print(len(words3),len(set(words3)))

print('Number of byes received:', len(src))

                   0   1
0                The   4
1                 UK   3
2                  s   9
3               data   5
4          regulator   1
5                 is   4
6            writing   1
7                 to  25
8           WhatsApp  15
9             demand   1
10              that  13
11               the  43
12              chat   1
13               app   3
14              does   3
15               not   9
16              hand   2
17              user   5
18          Facebook   9
19                as   4
20          millions   2
21         worldwide   1
22          continue   1
23              sign   1
24                up   2
25               for   5
26      alternatives   1
27              such   1
28            Signal   5
29               and  15
30          Telegram   3
31             avoid   1
32       forthcoming   1
33           changes   2
34               its   2
35             terms   3
36                of  12
37           service   3
38         Elizabeth   1


The API returns JSON, so we parse this using the in-built JSON library. 
The API specifies that all data are returned within the `response` key, even under failure. 
Thereofre, I have immediately descended to the response field 

# Parsing the JSON

In [9]:
response = json.loads(src)['response']
print('The following are available:\n ', sorted(response.keys()))

The following are available:
  ['currentPage', 'orderBy', 'pageSize', 'pages', 'results', 'startIndex', 'status', 'total', 'userTier']


# Verifying the status code

It is important to verify that the status message is `ok` before continuing - if it is not `ok` no 'real' data 
will have been received. 

In [10]:
assert response['status'] == 'ok'

# Listing the results 

The API standard states that the results will be found in the `results` field under the `response` field. 
Furthermore, the URLs will be found in the `webUrl` field, and the title will be found in the `webTitle` 
field. 

First let's look to see what a single result looks like in full, and then I will print a restricted 
set of parameters on the full set of results .

In [11]:
print(json.dumps(response['results'][0], indent=2, sort_keys=True))

{
  "apiUrl": "https://content.guardianapis.com/technology/2021/jan/27/facebook-earnings-stocks-revenue-fourth-quarter",
  "id": "technology/2021/jan/27/facebook-earnings-stocks-revenue-fourth-quarter",
  "isHosted": false,
  "pillarId": "pillar/news",
  "pillarName": "News",
  "sectionId": "technology",
  "sectionName": "Technology",
  "type": "article",
  "webPublicationDate": "2021-01-27T22:12:01Z",
  "webTitle": "Facebook CEO Mark Zuckerberg says platform will halt political suggestions",
  "webUrl": "https://www.theguardian.com/technology/2021/jan/27/facebook-earnings-stocks-revenue-fourth-quarter"
}


In [12]:
for result in response['results']: 
    print(result['webUrl'][:70], result['webTitle'][:20])

https://www.theguardian.com/technology/2021/jan/27/facebook-earnings-s Facebook CEO Mark Zu
https://www.theguardian.com/technology/2021/jan/26/uk-regulator-to-wri UK regulator to writ
https://www.theguardian.com/technology/2021/jan/26/grindr-fined-norway Grindr fined £8.6m i
https://www.theguardian.com/technology/2021/jan/25/google-announces-pl Google announces pla
https://www.theguardian.com/technology/2021/jan/24/whatsapp-loses-mill WhatsApp loses milli
https://www.theguardian.com/technology/2021/jan/24/is-it-time-to-leave Is it time to leave 
https://www.theguardian.com/technology/2021/jan/21/samsung-galaxy-s21- Samsung Galaxy S21 U
https://www.theguardian.com/technology/2021/jan/21/facebook-admits-enc Facebook admits encr
https://www.theguardian.com/technology/2021/jan/20/facebook-under-pres Facebook under press
https://www.theguardian.com/technology/2021/jan/19/facebook-asks-to-ap Facebook claims it d
https://www.theguardian.com/technology/2021/jan/16/should-you-keep-usi Should yo